In [ ]:
import os, sys
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
import cv2
from tqdm import tqdm
from collections import defaultdict
import random

In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)
from lib.FileLocationManager import DATA_PATH
from Controllers.SqlController import SqlController
color_radius = 3
animal = 'MD594'
sqlController = SqlController(animal)
ATLAS = 'atlasV8'
CSVPATH = os.path.join(DATA_PATH, 'atlas_data', ATLAS, animal)
jsonpath = os.path.join(CSVPATH,  'aligned_padded_structures.json')
print(jsonpath)
with open(jsonpath) as f:
    aligned_dict = json.load(f)

In [ ]:
structure_dict = sqlController.get_structures_dict()
SC = 'SC'
structure_dict[SC]

In [ ]:
#structure = random.choice(list(aligned_dict.keys()))
structure = 'SC'
onestructure = aligned_dict[structure]

In [ ]:
sections = [int(i) for i in onestructure.keys()]
section = random.choice(sections)
#section = 159
points = onestructure[str(section)]
arr = np.array(np.round(points)).astype(np.int32)
np.mean(arr, axis=0)

In [ ]:
INPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/thumbnail_aligned'
ORIGINPATH = '/net/birdstore/Active_Atlas_Data/data_root/CSHL/CSHL_volumes/MD585/10.0um_annotationAsScoreVolume'

In [ ]:
filepath = os.path.join(INPUT, f'{section}.tif')
img = cv2.imread(filepath)
for s, points in onestructure.items():
    if str(s) == str(section):
        arr = np.array(np.round(points)).astype(np.int32)
        color = sqlController.get_structure_color_rgb(structure)
        cv2.fillPoly(img, pts = [arr], color = color)


fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title(f'{structure} on {animal} aligned section:{section}', fontsize=20)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()

In [ ]:
title = []
filepath = os.path.join(INPUT, f'{section}.tif')
print(filepath)
img = cv2.imread(filepath)
for structure, data in aligned_dict.items():
    for s, points in data.items():
        if str(s) == str(section):
            title.append(structure)
            arr = np.array(np.round(points)).astype(np.int32)
            color = sqlController.get_structure_color_rgb(structure)
            cv2.polylines(img, [arr], isClosed=True, color=color, thickness=3)
            cv2.fillPoly(img, pts = [arr], color = color)
            com = np.mean(arr, axis=0)
            cv2.circle(img, (int(com[0]), int(com[1])), 5, (255,255,200), 2)


fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title(f'{title} on {animal} aligned section:{section}', fontsize=20)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()